# Transit On the SHN 
* [GH Issue](https://github.com/cal-itp/data-analyses/issues/1477)


In [ ]:
import geopandas as gpd
import google.auth
import numpy as np
import pandas as pd

credentials, project = google.auth.default()

import gcsfs

fs = gcsfs.GCSFileSystem()

In [ ]:
from calitp_data_analysis import geography_utils, utils
from segment_speed_utils import gtfs_schedule_wrangling, helpers
from shared_utils import (
    catalog_utils,
    dask_utils,
    gtfs_utils_v2,
    portfolio_utils,
    publish_utils,
    rt_dates,
    rt_utils,
)
from update_vars import GTFS_DATA_DICT, RT_SCHED_GCS, SCHED_GCS, SEGMENT_GCS

In [ ]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [ ]:
def process_transit_routes() -> gpd.GeoDataFrame:
    """
    Select the most recent transit route to
    figure out how much of it intersects with
    the state highway network.
    """
    # Load in the route shapes.
    OPERATOR_ROUTE = GTFS_DATA_DICT.digest_tables.operator_routes_map

    subset = [
        "service_date",
        "geometry",
        "portfolio_organization_name",
        "recent_combined_name",
        # "route_id",
    ]
    op_geography_df = gpd.read_parquet(
        f"{RT_SCHED_GCS}{OPERATOR_ROUTE}.parquet",
        storage_options={"token": credentials.token},
    )[subset]

    # Keep the row for each portfolio_organization_name/recent_combined_name
    # that is the most recent.
    most_recent_routes = publish_utils.filter_to_recent_date(
        df=op_geography_df,
        group_cols=[
            "portfolio_organization_name",
        ],
    )

    # Calculate the length of route, ensuring that it is in feet.
    most_recent_routes = most_recent_routes.assign(
        route_length_feet=most_recent_routes.geometry.to_crs(
            geography_utils.CA_NAD83Albers_ft
        ).length
    )

    # Drop any duplicates.
    # This will probably be taken out once the 1:m recent_combined_name
    # to route_id issue is resolved.
    most_recent_routes = most_recent_routes.drop_duplicates(
        subset=["portfolio_organization_name", "recent_combined_name", "service_date"]
    )
    return most_recent_routes

In [ ]:
# transit_routes = process_transit_routes()

In [ ]:
# transit_routes.drop(columns=["geometry"]).sample(3)

In [ ]:
# len(transit_routes)

In [ ]:
# transit_routes.recent_combined_name.value_counts().head()

In [ ]:
# transit_routes.portfolio_organization_name.value_counts().head()

## Load in SHS 


In [ ]:
def dissolve_shn(columns_to_dissolve: list, file_name: str) -> gpd.GeoDataFrame:
    """
    Dissolve State Highway Network so there will only be one row for each
    route name and route type
    """
    # Read in the dataset and change the CRS to one to feet.
    SHN_FILE = catalog_utils.get_catalog(
        "shared_data_catalog"
    ).state_highway_network.urlpath

    shn = gpd.read_parquet(
        SHN_FILE,
        storage_options={"token": credentials.token},
    ).to_crs(geography_utils.CA_NAD83Albers_ft)

    # Dissolve by route which represents the the route's name and drop the other columns
    # because they are no longer relevant.
    shn_dissolved = (shn.dissolve(by=columns_to_dissolve).reset_index())[
        columns_to_dissolve + ["geometry"]
    ]

    # Rename because I don't want any confusion between SHN route and
    # transit route.
    shn_dissolved = shn_dissolved.rename(columns={"Route": "shn_route"})

    # Find the length of each highway.
    shn_dissolved = shn_dissolved.assign(
        highway_feet=shn_dissolved.geometry.length,
        shn_route=shn_dissolved.shn_route.astype(int),
    )

    # Save this out so I don't have to dissolve it each time.
    shn_dissolved.to_parquet(
        f"gs://calitp-analytics-data/data-analyses/state_highway_network/shn_dissolved_by_{file_name}.parquet",
        filesystem=fs,
    )
    return shn_dissolved

In [ ]:
# dissolved_route = dissolve_shn(["Route","District"], "ct_district_route")

In [ ]:
# shn_dissolved.loc[shn_dissolved.shn_route == 210].drop(columns=["geometry"])

In [ ]:
# shn_dissolved.loc[shn_dissolved.shn_route == 110].drop(columns=["geometry"])

In [ ]:
# shn_dissolved.loc[shn_dissolved.Route == 210].explore()

In [ ]:
# shn_dissolved.loc[shn_dissolved.Route == 110].explore()

In [ ]:
# len(dissolved)

In [ ]:
def buffer_shn(buffer_amount: int, file_name: str) -> gpd.GeoDataFrame:
    """
    Add a buffer to the SHN before overlaying it with
    transit routes.
    """
    GCS_FILE_PATH = "gs://calitp-analytics-data/data-analyses/state_highway_network/"

    # Read in the dissolved SHN file
    shn_df = gpd.read_parquet(
        f"{GCS_FILE_PATH}shn_dissolved_by_{file_name}.parquet",
        storage_options={"token": credentials.token},
    )

    # Buffer the state highway.
    shn_df_buffered = shn_df.assign(
        geometry=shn_df.geometry.buffer(buffer_amount),
    )

    # Save it out so we won't have to buffer over again and
    # can just read it in.
    shn_df_buffered.to_parquet(
        f"{GCS_FILE_PATH}shn_buffered_{buffer_amount}_ft_{file_name}.parquet",
        filesystem=fs,
    )

    return shn_df_buffered

In [ ]:
SHN_HWY_BUFFER_FEET = 50
PARALLEL_HWY_BUFFER_FEET = geography_utils.FEET_PER_MI * 0.5

In [ ]:
# intersecting_buffer.shape

In [ ]:
# buffered_df = buffer_shn(SHN_HWY_BUFFER_FEET, "ct_district_route")

In [ ]:
# buffered_df.loc[buffered_df.shn_route == 5].explore()

In [ ]:
shn_district_df = gpd.read_parquet(
    f"gs://calitp-analytics-data/data-analyses/state_highway_network/shn_buffered_50_ft_ct_district_route.parquet",
    storage_options={"token": credentials.token},
)

In [ ]:
shn_district_df.shape

In [ ]:
# buffered_df.columns

## Overlay

In [ ]:
def routes_shn_intersection(buffer_amount: int, file_name: str) -> gpd.GeoDataFrame:
    """
    Overlay the most recent transit routes with a buffered version
    of the SHN
    """
    GCS_FILE_PATH = "gs://calitp-analytics-data/data-analyses/state_highway_network/"

    # Read in buffered shn here or re buffer if we don't have it available.
    HWY_FILE = f"{GCS_FILE_PATH}shn_buffered_{buffer_amount}_ft_{file_name}.parquet"

    if fs.exists(HWY_FILE):
        shn_routes_gdf = gpd.read_parquet(
            HWY_FILE, storage_options={"token": credentials.token}
        )
    else:
        shn_routes_gdf = buffer_shn(buffer_amount)

    # Process the most recent transit route geographies and ensure the
    # CRS matches the SHN routes' GDF so the overlay doesn't go wonky.
    transit_routes_gdf = process_transit_routes().to_crs(shn_routes_gdf.crs)

    # Overlay transit routes with the SHN geographies.
    gdf = gpd.overlay(
        transit_routes_gdf, shn_routes_gdf, how="intersection", keep_geom_type=True
    )

    # Calcuate the percent of the transit route that runs on a highway, round it up and
    # multiply it by 100. Drop the geometry because we want the original transit route
    # shapes.
    gdf = gdf.assign(
        pct_route_on_hwy=(gdf.geometry.length / gdf.route_length_feet).round(3) * 100,
    ).drop(
        columns=[
            "geometry",
        ]
    )

    # Join back the dataframe above with the original transit route dataframes
    # so we can have the original transit route geographies.
    gdf2 = pd.merge(
        transit_routes_gdf,
        gdf,
        on=[
            "service_date",
            "portfolio_organization_name",
            "recent_combined_name",
            "route_length_feet",
        ],
        how="left",
    )

    return gdf2

In [ ]:
intersecting = routes_shn_intersection(SHN_HWY_BUFFER_FEET, "ct_district_route")

In [ ]:
intersecting.pct_route_on_hwy.describe()

In [ ]:
intersecting.recent_combined_name.value_counts().describe()

In [ ]:
# Find routes that cross multiple districts
multi_district_routes = (
    intersecting.groupby(["portfolio_organization_name", "recent_combined_name"])
    .agg({"District": "nunique"})
    .reset_index()
)

In [ ]:
multi_district_routes.sort_values(by=["District"], ascending=False).head(20)

In [ ]:
bakersfield_route = intersecting.loc[
    intersecting.recent_combined_name == "100 Bakersfield - Lancaster"
]

In [ ]:
bakersfield_route.District.unique()

In [ ]:
# bakersfield_route.drop(columns = ["service_date"]).explore()

In [ ]:
intersecting.loc[
    intersecting.recent_combined_name == "100 Bakersfield - Lancaster"
].pct_route_on_hwy.sum()

## Create final dataframe to add to Transit Routes dataset on the Open Data Portal [here](https://data.ca.gov/dataset/california-transit-routes/resource/3ea7eb11-4fc6-45ed-88f5-b599e38c6b19)

In [ ]:
def group_route_district(df: pd.DataFrame, pct_route_on_hwy_agg: str) -> pd.DataFrame:
    agg1 = (
        df.groupby(
            [
                "portfolio_organization_name",
                "recent_combined_name",
            ],
            as_index=False,
        )[["shn_route", "District", "pct_route_on_hwy"]]
        .agg(
            {
                "shn_route": lambda x: ", ".join(set(x.astype(str))),
                "District": lambda x: ", ".join(set(x.astype(str))),
                "pct_route_on_hwy": pct_route_on_hwy_agg,
            }
        )
        .reset_index(drop=True)
    )
    return agg1

In [ ]:
# group_route_district()

In [ ]:
def create_on_shs_column(df):
    df["on_shs"] = np.where(df["pct_route_on_hwy"] == 0, "N", "Y")
    return df

In [ ]:
def prep_open_data_portal(gdf: gpd.GeoDataFrame) -> pd.DataFrame:
    """
    Prepare the gdf to join with the existing transit_routes
    dataframe that is published on the Open Data Portal
    """
    # Group the dataframe so that one route only has one
    # row instead of multiple rows after finding its
    # intersection with any SHN routes.
    # Sum up the percentage of a transit route that
    # intersects with the highway.
    agg1 = group_route_district(gdf, "sum")

    # Add yes/no column to signify if a transit route intersects
    # with a SHN route
    agg1 = create_on_shs_column(agg1)
    return agg1

In [ ]:
open_data_portal_df = prep_open_data_portal(intersecting)

In [ ]:
open_data_portal_df.on_shs.value_counts()

In [ ]:
open_data_portal_df.loc[open_data_portal_df.on_shs == "Y"].pct_route_on_hwy.describe()

In [ ]:
open_data_portal_df.loc[open_data_portal_df.on_shs == "N"].pct_route_on_hwy.describe()

In [ ]:
open_data_portal_df.sample(10)

## Create final dataframes for portfolio

In [ ]:
open_data_portal_df.District.unique()

In [ ]:
def categorize_percentiles(df):
    bins = [20, 41, 61, 81, np.inf]
    labels = ['20-40', '41-60', '61-80', '81-100']
    df['percentile_route'] = pd.cut(df['pct_route_on_hwy'], 
                                    bins=bins, labels=labels, right=False).astype(str)
    return df

In [ ]:
def final_transit_route_shs_outputs(
    intersecting_gdf: gpd.GeoDataFrame,
    open_data_df: pd.DataFrame,
    pct_route_intersection: int,
    district: str,
):
    """
    Take the dataframes from prep_open_data_portal and routes_shn_intersection.
    Prepare them for display on the GTFS Caltrans District Digest.

    intersecting_gdf: geodataframe created by
    open_data_df: dataframe created by
    pct_route_intersection: cutoff of the % of the transit route intersecting with the SHN
    district: the Caltrans district we are interested in.
    """
    # Filter out for any pct_route_on_hwy that we deem too low & for the relevant district.
    open_data_df = open_data_df.loc[
        (open_data_df.pct_route_on_hwy > pct_route_intersection)
        & (open_data_df.District.str.contains(district))
    ]
    intersecting_gdf.District = intersecting_gdf.District.astype(str)
    intersecting_gdf = intersecting_gdf.loc[
        intersecting_gdf.District.str.contains(district)
    ]

    # Join back to get the original transit route geometries and the names of the
    # state highways these routes intersect with. This gdf will be used to
    # display a map.
    map_gdf = pd.merge(
        intersecting_gdf[
            ["portfolio_organization_name", "recent_combined_name", "geometry"]
        ].drop_duplicates(),
        open_data_df,
        on=["portfolio_organization_name", "recent_combined_name"],
    )

    # Add column for color scale when mapping 
    map_gdf = categorize_percentiles(map_gdf)
    # We want a text table to display.
    # Have to rejoin and to find only the SHN routes that are in the district
    # we are interested in.
    text_table_df = pd.merge(
        intersecting_gdf[
            [
                "portfolio_organization_name",
                "recent_combined_name",
                "shn_route",
                "District",
            ]
        ],
        open_data_df[
            ["portfolio_organization_name", "recent_combined_name", "pct_route_on_hwy"]
        ],
        on=["portfolio_organization_name", "recent_combined_name"],
    )

    # Now we have to aggregate again so each route will only have one row with the
    # district and SHN route info delinated by commas if there are multiple values.
    text_table = group_route_district(text_table_df, "max")

    # Rename for clarity
    text_table = text_table.rename(
        columns={
            "shn_route": f"shn_routes_in_d_{district}",
            "pct_route_on_hwy": "pct_route_on_hwy_across_districts",
        }
    )
    return map_gdf, text_table

### Example using 100 Bakersfield - Lancaster which crosses D6, D7, D9

In [ ]:
map_gdf_d7, text_df_d7 = final_transit_route_shs_outputs(
    intersecting, open_data_portal_df, 20, "7"
)

In [ ]:
map_gdf_d9, text_df_d9 = final_transit_route_shs_outputs(
    intersecting, open_data_portal_df, 20, "9"
)

In [ ]:
text_df_d7.loc[text_df_d7.recent_combined_name == "100 Bakersfield - Lancaster"]

In [ ]:
text_df_d6.loc[text_df_d6.recent_combined_name == "100 Bakersfield - Lancaster"]

In [ ]:
text_df_d9.loc[text_df_d9.recent_combined_name == "100 Bakersfield - Lancaster"]

## Sample Map
* This will be displayed by each Caltrans District as opposed to this sample which shows all the routes across the state that touch a state highway.

* Read in the buffered SHN file and dissolve it again so it's only one row, since we don't care about each individual state route.
* Amanda, note to self: should save this out to GCS because there's no point in dissolving over & over again
* Bug in geopandas [documented here](https://github.com/geopandas/geopandas/issues/3194), you can't specify a column and color.

In [ ]:
GCS_FILE_PATH = "gs://calitp-analytics-data/data-analyses/state_highway_network/"

# Read in buffered shn here or re buffer
HWY_FILE = (
    f"{GCS_FILE_PATH}shn_buffered_{SHN_HWY_BUFFER_FEET}_ft_ct_district_route.parquet"
)

In [ ]:
shn_gdf = gpd.read_parquet(HWY_FILE, storage_options={"token": credentials.token})

In [ ]:
len(shn_gdf)

In [ ]:
shn_gdf2 = shn_gdf.dissolve("District").reset_index()

In [ ]:
len(shn_gdf2)

In [ ]:
shn_gdf2.columns

In [ ]:
shn_gdf2.District.unique()

In [ ]:
m = shn_gdf2.loc[shn_gdf2.District == 7].explore(
    height=500,
    width=1000,
    style_kwds={"color": "#9DA4A6", "weight": 6, "opacity": 0.5},
    tiles="CartoDB positron",
    name="shs",
)

In [ ]:
cmap_colors = [ "#93c3db", "#144c87", "#8c1024", "#e4846c",]

In [ ]:
map_gdf_d7.explore("percentile_route", m=m, categorical = True, cmap = cmap_colors, legend = True)

### Sample Text Table

In [ ]:
text_df_d7.sort_values(by = ["pct_route_on_hwy_across_districts"], ascending = False).head()